# Zadanie 2: Obliczanie macheps metodą Kahana

## Część 2.1: Weryfikacja formuły Kahana

### 1. Opis problemu
Profesor William Kahan zasugerował, że epsilon maszynowy ($macheps$) można wyznaczyć w arytmetyce zmiennoprzecinkowej poprzez obliczenie wyrażenia:
$3 \times (4/3 - 1) - 1$.

Celem tego zadania jest eksperymentalna weryfikacja tego stwierdzenia w języku Julia dla typów `Float16`, `Float32` i `Float64`. Porównamy wynik tej formuły z wartością zwracaną przez wbudowaną funkcję `eps(T)`, która (jak ustaliliśmy w Zadaniu 1) w Julii zwraca $macheps$, czyli `eps(T(1.0))`.

In [1]:
using Printf

"""
Oblicza epsilon maszynowy używając formuły Kahana
3 * (4/3 - 1) - 1
dla zadanego typu zmiennoprzecinkowego T.
"""
function kahan_macheps(T::Type{<:AbstractFloat})
    # Definiujemy stałe *wewnątrz* arytmetyki typu T
    three::T = T(3.0)
    four::T = T(4.0)
    one::T = T(1.0)
    
    # Krok 1: Oblicz 4/3 (kluczowe zaokrąglenie)
    four_thirds::T = four / three
    
    # Krok 2: Oblicz (4/3 - 1)
    one_third_approx::T = four_thirds - one
    
    # Krok 3: Oblicz 3 * (4/3 - 1)
    one_approx::T = three * one_third_approx
    
    # Krok 4: Oblicz 3 * (4/3 - 1) - 1 (izolacja błędu)
    result::T = one_approx - one
    
    return result
end

# Lista typów do przetestowania
types_to_test = [Float16, Float32, Float64]
println("--- Sprawdzanie formuły Kahana dla macheps ---")

for T in types_to_test
    kahan_val = kahan_macheps(T)
    built_in_eps = eps(T) # eps(T) w Julii to eps(T(1.0)), czyli macheps
    
    # Używamy formatowania %.10e dla zwiększenia precyzji
    @printf "--- Typ: %s ---\n" T
    @printf "Wynik z formuły Kahana: \t%.10e\n" kahan_val
    @printf "Wbudowane (eps(T)): \t\t%.10e\n" built_in_eps
    @printf "Czy równe eps(T)? \t\t%s\n" kahan_val == built_in_eps
    @printf "Czy równe -eps(T)? \t\t%s\n" kahan_val == -built_in_eps
end

--- Sprawdzanie formuły Kahana dla macheps ---
--- Typ: Float16 ---
Wynik z formuły Kahana: 	-9.7656250000e-04
Wbudowane (eps(T)): 		9.7656250000e-04
Czy równe eps(T)? 		false
Czy równe -eps(T)? 		true
--- Typ: Float32 ---
Wynik z formuły Kahana: 	1.1920928955e-07
Wbudowane (eps(T)): 		1.1920928955e-07
Czy równe eps(T)? 		true
Czy równe -eps(T)? 		false
--- Typ: Float64 ---
Wynik z formuły Kahana: 	-2.2204460493e-16
Wbudowane (eps(T)): 		2.2204460493e-16
Czy równe eps(T)? 		false
Czy równe -eps(T)? 		true


### 3. Wyniki i interpretacja
**Interpretacja:**
Eksperyment dał zaskakujący i bardzo pouczający wynik. Wartość bezwzględna formuły Kahana jest zawsze równa `eps(T)`, jednak znak wyniku zależy od typu zmiennoprzecinkowego.

* **Float16:**
    * Wynik Kahana: `-9.7656250000e-04`
    * Wbudowane `eps(T)`: `9.7656250000e-04`
    * Zgodność: `kahan_val == -eps(T)` (Prawda)

* **Float32:**
    * Wynik Kahana: `1.1920928955e-07`
    * Wbudowane `eps(T)`: `1.1920928955e-07`
    * Zgodność: `kahan_val == eps(T)` (Prawda)

* **Float64:**
    * Wynik Kahana: `-2.2204460493e-16`
    * Wbudowane `eps(T)`: `2.2204460493e-16`
    * Zgodność: `kahan_val == -eps(T)` (Prawda)

### 4. Wnioski
Stwierdzenie Kahana jest słuszne co do *wartości bezwzględnej* – formuła ta doskonale izoluje błąd zaokrąglenia o wielkości $macheps$. Znak wyniku jest fascynującą ilustracją subtelności reguł zaokrąglania w **standardzie IEEE 754**.

Kluczowa jest pierwsza operacja: $fl(4/3)$.
Liczba $4/3$ ma nieskończone rozwinięcie binarne: $1.01010101..._2$.
Standard **IEEE 754** stosuje zaokrąglanie do najbliższej wartości ("round-to-nearest-ties-to-even").

1.  **Przypadek `Float16` ($p=11$ bitów mantysy) i `Float64` ($p=53$ bity):**
    * `Float16` (mantysa `1.0101010101`): 12-ty bit (pierwszy odcięty) to `0`. Część odcięta $0.0101..._2$ jest mniejsza niż połowa. Liczba jest zaokrąglana **w dół**.
    * `Float64` (mantysa `1.0101...01`): 54-ty bit to `0`. Liczba jest zaokrąglana **w dół**.
    * **Analiza błędu (Round-Down):**
        * $fl(4/3) = 4/3 - \epsilon_1$ (błąd początkowy jest ujemny)
        * $fl(fl(4/3) - 1) \approx 1/3 - \epsilon_1$
        * $fl(3 \cdot ...) \approx 1 - 3\epsilon_1$
        * $fl(...) - 1 \approx (1 - 3\epsilon_1) - 1 = -3\epsilon_1$
    * Izolowany błąd to $-macheps$. Wynik jest **ujemny**.

2.  **Przypadek `Float32` ($p=24$ bity mantysy):**
    * `Float32` (mantysa `1.01010101010101010101010`): 25-ty bit (pierwszy odcięty) to `1`. Część odcięta $1.0101..._2$ jest większa niż połowa. Liczba jest zaokrąglana **w górę**.
    * **Analiza błędu (Round-Up):**
        * $fl(4/3) = 4/3 + \epsilon_1$ (błąd początkowy jest dodatni)
        * $fl(fl(4/3) - 1) \approx 1/3 + \epsilon_1$
        * $fl(3 \cdot ...) \approx 1 + 3\epsilon_1$
        * $fl(...) - 1 \approx (1 + 3\epsilon_1) - 1 = +3\epsilon_1$
    * Izolowany błąd to $+macheps$. Wynik jest **dodatni**.

Eksperyment ten pokazuje, że kierunek zaokrąglenia w **obliczeniach maszynowych** zależy od konkretnej liczby bitów precyzji, co może prowadzić do pozornie sprzecznych (ale poprawnych) wyników dla różnych typów zmiennoprzecinkowych.